In [ ]:
# %reload_ext autoreload
# %autoreload 2
import imageio
import matplotlib.pyplot as plt
import torch
print(torch.cuda.is_available())
import sys
sys.path.append("./GLIP/")

from meshseg.models.GLIP.glip import GLIPModel
GM = GLIPModel()

In [ ]:
si = 128
# img = torch.zeros((si, si, 3), dtype=torch.uint8).numpy()
img = imageio.imread("data/img/table.jpg")
# img += 2
# plt.imshow(img)
print(img.shape)
# prediction = GM.predict(img, "The legs, head, eyes, arms of people")
# prediction = GM.predict(img, "the legs, top of the table")
prediction = GM.predict(img, "legs")

In [ ]:
print(prediction[1])
# plt.figure(figsize=[15, 10])
plt.figure(figsize=[8, 6])
plt.imshow(prediction[0])
plt.show()